In [1]:
import shutil
import os
import yaml
import json
from sklearn.model_selection import train_test_split

In [2]:
# Define your source directories
source_images_dir = '/kaggle/input/customdataset/ObjectDetection/images/train'
yolo_annotations_dir = '/kaggle/input/customdataset/ObjectDetection/labels/train'
coco_annotations_path = '/kaggle/input/coco-annotations/instances_default.json'

# Define the output base directories
base_dir = '/kaggle/working'
images_dir = os.path.join(base_dir, 'images')
labels_dir = os.path.join(base_dir, 'labels')
coco_dir = os.path.join(base_dir, 'coco_annotations')

# Make sure the base directories exist
os.makedirs(images_dir, exist_ok=True)
os.makedirs(labels_dir, exist_ok=True)
os.makedirs(coco_dir, exist_ok=True)

# Prepare subdirectories for train and val in each base directory
for sub_dir in ['train', 'val']:
    os.makedirs(os.path.join(images_dir, sub_dir), exist_ok=True)
    os.makedirs(os.path.join(labels_dir, sub_dir), exist_ok=True)

# List and split image files
image_files = [f for f in os.listdir(source_images_dir) if f.endswith('.jpg')]
train_files, val_files = train_test_split(image_files, test_size=0.3, random_state=42)

# Load COCO annotations
with open(coco_annotations_path) as f:
    coco_annotations = json.load(f)

def filter_coco_annotations(files, annotations):
    filtered = {'images': [], 'annotations': [], 'categories': annotations['categories']}
    image_ids = {os.path.splitext(f)[0] for f in files}
    filtered['images'] = [image for image in annotations['images'] if image['file_name'].replace('.jpg', '') in image_ids]
    filtered_image_ids = {image['id'] for image in filtered['images']}
    filtered['annotations'] = [ann for ann in annotations['annotations'] if ann['image_id'] in filtered_image_ids]
    return filtered

def organize_dataset(files, source_images_dir, yolo_dir, images_dest_dir, labels_dest_dir):
    for filename in files:
        # Copy image
        shutil.copy(os.path.join(source_images_dir, filename), os.path.join(images_dest_dir, filename))
        
        # Copy YOLO annotation
        yolo_annotation_filename = filename.replace('.jpg', '.txt')
        shutil.copy(os.path.join(yolo_dir, yolo_annotation_filename), os.path.join(labels_dest_dir, yolo_annotation_filename))

# Filter COCO annotations for train and val sets
train_annotations_coco = filter_coco_annotations(train_files, coco_annotations)
val_annotations_coco = filter_coco_annotations(val_files, coco_annotations)

# Organize train and validation datasets
organize_dataset(train_files, source_images_dir, yolo_annotations_dir, os.path.join(images_dir, 'train'), os.path.join(labels_dir, 'train'))
organize_dataset(val_files, source_images_dir, yolo_annotations_dir, os.path.join(images_dir, 'val'), os.path.join(labels_dir, 'val'))

# Save filtered COCO annotations
with open(os.path.join(coco_dir, 'train.json'), 'w') as f:
    json.dump(train_annotations_coco, f)
with open(os.path.join(coco_dir, 'val.json'), 'w') as f:
    json.dump(val_annotations_coco, f)

print("Dataset organized with separate directories for images, YOLO labels, and COCO annotations.")

Dataset organized with separate directories for images, YOLO labels, and COCO annotations.


In [3]:
# The path to the original YAML file
input_yaml_path = '/kaggle/input/customdataset/ObjectDetection/google_colab_files.yaml'
# The path where the modified YAML file will be saved
output_yaml_path = '/kaggle/working/google_colab_files.yaml'

# Load the original YAML file
with open(input_yaml_path, 'r') as file:
    data = yaml.safe_load(file)

# Modify the dataset configuration as needed
data['path'] = '/kaggle/working'
data['val'] = 'images/val'  # Assuming you want to point to the validation dataset

# Save the modified configuration to a new YAML file
with open(output_yaml_path, 'w') as file:
    yaml.dump(data, file, sort_keys=False)  # sort_keys=False to preserve the order of keys

In [4]:
!pip install pycocotools
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.2/426.2 kB 6.4 MB/s eta 0:00:00a 0:00:01
  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-7sps9zdo
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-7sps9zdo
  Resolved https://github.com/facebookresearch/detectron2.git to commit eb96ee1d4752ff5896f623f738641fba9c755237
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.

In [5]:
from detectron2.data.datasets import register_coco_instances

register_coco_instances("coco_dataset_train", {}, "/kaggle/working/coco_annotations/train.json", "/kaggle/working/images/train")
register_coco_instances("coco_dataset_val", {}, "/kaggle/working/coco_annotations/val.json", "/kaggle/working/images/val")

In [6]:
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer

cfg = get_cfg()
# Use model_zoo to get the path for the config file
config_path = model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
cfg.merge_from_file(config_path)

cfg.DATASETS.TRAIN = ("coco_dataset_train",)
cfg.DATASETS.TEST = ("coco_dataset_val",)
cfg.DATALOADER.NUM_WORKERS = 2

# Let training initialize from model zoo
cfg.MODEL.WEIGHTS = "detectron2://ImageNetPretrained/MSRA/R-50.pkl"  # Initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.0025
cfg.SOLVER.MAX_ITER = 1000   # Adjust according to your dataset size
cfg.OUTPUT_DIR = '/kaggle/working/results'
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

R-50.pkl: 102MB [00:03, 26.6MB/s]                              
/opt/conda/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /usr/local/src/pytorch/aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
2024-03-24 16:49:23.266426: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-24 16:49:23.266530: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-24 16:49:23.373815: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when o

In [10]:
from detectron2.data import build_detection_test_loader
from detectron2.evaluation import inference_on_dataset
from detectron2.evaluation import COCOEvaluator
import time
import torch 
# Ensure the model is in evaluation mode
trainer.model.eval()

# Load the validation dataset
val_loader = build_detection_test_loader(cfg, "coco_dataset_val")

In [11]:
total_time = 0
total_images = 0

# Perform inference and measure time per image
for idx, inputs in enumerate(val_loader):
    start_time = time.time()
    with torch.no_grad():
        predictions = trainer.model(inputs)
    end_time = time.time()
    total_time += (end_time - start_time)
    total_images += len(inputs)

# Calculate average inference time per image
average_time_per_image = total_time / total_images
print(f"Average inference time per image: {average_time_per_image:.4f} seconds")

Average inference time per image: 0.0732 seconds


In [12]:
# Create a COCOEvaluator instance for the validation dataset
evaluator = COCOEvaluator("coco_dataset_val", cfg, False, output_dir="./output/")

total_time = 0
total_images = 0

# Run inference on the dataset and evaluate the results using the evaluator
print("Evaluating model on COCO validation dataset...")
evaluation_results = inference_on_dataset(trainer.model, val_loader, evaluator)

# Print evaluation results
print(evaluation_results)

Evaluating model on COCO validation dataset...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.202
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.460
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.169
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.202
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.180
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.296
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.302
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100

In [1]:
import json

# Path to the COCO annotations file
annotations_file = '/kaggle/input/coco-annotations/instances_default.json'

# Load the COCO annotations
with open(annotations_file, 'r') as f:
    coco_data = json.load(f)

# Initialize a dictionary to hold the count of each category
category_counts = {}

# Iterate over all annotations to count the instances per category
for annotation in coco_data['annotations']:
    category_id = annotation['category_id']
    if category_id in category_counts:
        category_counts[category_id] += 1
    else:
        category_counts[category_id] = 1

# Translate category IDs to category names (for readability)
category_names = {category['id']: category['name'] for category in coco_data['categories']}
category_counts_named = {category_names[cat_id]: count for cat_id, count in category_counts.items()}

# Print the count of instances for each category
for category, count in category_counts_named.items():
    print(f'Category: {category}, Instances: {count}')


Category: drink, Instances: 255
Category: utensil, Instances: 144
Category: laptop, Instances: 138
